# Add Classifier columns
* Dusk, Dawn, Night
* Twilight

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../src/fall2017_aug1_oct31_tseep_thrush_original.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189983 entries, 0 to 189982
Data columns (total 23 columns):
season                   189983 non-null object
year                     189983 non-null int64
detector                 189983 non-null object
species                  188791 non-null object
site                     189983 non-null object
date                     189983 non-null object
recording_start          189983 non-null object
recording_length         189983 non-null object
detection_time           189983 non-null object
real_detection_time      189983 non-null object
real_detection_time.1    189983 non-null object
rounded_to_half_hour     189983 non-null object
duplicate                187890 non-null object
sunset                   189983 non-null object
civil_dusk               189983 non-null object
nautical_dusk            189983 non-null object
astronomical_dusk        189983 non-null object
astronomical_dawn        189983 non-null object
nautical_dawn            1

In [5]:
df.real_detection_time.head(2)

0    04:41:19
1    04:41:28
Name: real_detection_time, dtype: object

In [6]:
date_columns = ['real_detection_time.1', 'sunset', 'civil_dusk', 'nautical_dusk', 'astronomical_dusk',\
          'astronomical_dawn', 'nautical_dawn', 'civil_dawn', 'sunrise']

### Assign local datetime type

In [ ]:
for column in date_columns:
    print('Starting....')
    df[column] = pd.to_datetime(df.loc[:, column]).dt.tz_localize('US/Mountain')
    print(column + ' complete')

## Time periods
### Dusk and Dawn

* 'sunset' <= civil_twilight, dusk < 'civil_dusk'
* 'civil_dusk' <= nautical_twilight, dusk < 'nautical_dusk'
* 'nautical_dusk' <= astronomical_twilight, dusk < 'astronomical_dusk'
* 'astronomical_dusk' <= night, night < 'astronomical_dawn'
* 'astronomical_dawn' <= astronomical_twilight, dawn < 'nautical_dawn'
* 'nautical_dawn' <= nautical_twilight, dawn < 'civil_dawn'
* 'civil_dawn' <= civil_twilight, dawn < 'sunrise'

### Columns

* twilight
* dusk_dawn

### Function for classifying detection time

In [9]:
def dusk_dawn(row):
    detection = row['real_detection_time.1']
    
    if row['sunset'] <= detection < row['civil_dusk']:
        return 'civil_twilight, dusk'
    elif row['civil_dusk'] <= detection < row['nautical_dusk']:
        return 'nautical_twilight, dusk'
    elif row['nautical_dusk'] <= detection < row['astronomical_dusk']:
        return 'astronomical_twilight, dusk'
    elif row['astronomical_dusk'] <= detection < row['astronomical_dawn']:
        return 'night, night'
    elif row['astronomical_dawn'] <= detection < row['nautical_dawn']:
        return 'astronomical_twilight, dawn'
    elif row['nautical_dawn'] <= detection < row['civil_dawn']:
        return 'nautical_twilight, dawn'
    elif row['civil_dawn'] <= detection < row['sunrise']:
        return 'civil_twilight, dawn'
    else:
        print('Row out of range')

In [10]:
df['dusk_dawn'] = df.apply(dusk_dawn, axis=1)

In [256]:
df = df.drop(['dusk_dawn'], axis=1)

In [265]:
df.loc[70:80, ['astronomical_dusk', 'real_detection_time.1', 'astronomical_dawn', 'dusk_dawn', 'twilight']];

In [234]:
df['twilight'], df['dusk_dawn_x'] = df['dusk_dawn'].str.split(', ', 1).str

In [267]:
df.twilight.value_counts()

night                    134300
nautical_twilight         36151
astronomical_twilight     17825
civil_twilight             1707
Name: twilight, dtype: int64

In [268]:
df.dusk_dawn.value_counts()

night    134300
dawn      50346
dusk       5337
Name: dusk_dawn, dtype: int64

In [269]:
df.to_csv('../output/fall2017_aug1_oct31_tseep_thrush_twilight.csv', index=False)